<img src="https://upload.wikimedia.org/wikipedia/commons/4/47/Logo_UTFSM.png" width="200" alt="utfsm-logo" align="left"/>

# MAT281
### Aplicaciones de la Matemática en la Ingeniería

## Módulo 04
## Laboratorio Clase 04: Métricas y selección de modelos

### Instrucciones


* Completa tus datos personales (nombre y rol USM) en siguiente celda.
* La escala es de 0 a 4 considerando solo valores enteros.
* Debes _pushear_ tus cambios a tu repositorio personal del curso.
* Como respaldo, debes enviar un archivo .zip con el siguiente formato `mXX_cYY_lab_apellido_nombre.zip` a alonso.ogueda@gmail.com, debe contener todo lo necesario para que se ejecute correctamente cada celda, ya sea datos, imágenes, scripts, etc.
* Se evaluará:
    - Soluciones
    - Código
    - Que Binder esté bien configurado.
    - Al presionar  `Kernel -> Restart Kernel and Run All Cells` deben ejecutarse todas las celdas sin error.

__Nombre__: IGNACIO JERIA MARRAS

__Rol__: 201684022-2

En este laboratorio utilizaremos el conjunto de datos _Abolone_. 

**Recuerdo**

La base de datos contiene mediciones a 4177 abalones, donde las mediciones posibles son sexo ($S$), peso entero $W_1$, peso sin concha $W_2$, peso de visceras $W_3$, peso de concha  $W_4$, largo ($L$), diametro $D$, altura $H$, y el número de anillos $A$. 

In [1]:
import pandas as pd
import numpy as np

In [2]:
abalone = pd.read_csv(
    "data/abalone.data",
    header=None,
    names=["sex", "length", "diameter", "height", "whole_weight", "shucked_weight", "viscera_weight", "shell_weight", "rings"]
)

abalone_data = (
    abalone.assign(sex=lambda x: x["sex"].map({"M": 1, "I": 0, "F": -1}))
    .loc[lambda x: x.drop(columns="sex").gt(0).all(axis=1)]
    .astype(np.float)
)
abalone_data.head()

,sex,length,diameter,height,whole_weight,shucked_weight,viscera_weight,shell_weight,rings
0,1.0,0.455,0.365,0.095,0.5140,0.2245,0.1010,0.150,15.0
1,1.0,0.350,0.265,0.090,0.2255,0.0995,0.0485,0.070,7.0
2,-1.0,0.530,0.420,0.135,0.6770,0.2565,0.1415,0.210,9.0
3,1.0,0.440,0.365,0.125,0.5160,0.2155,0.1140,0.155,10.0
4,0.0,0.330,0.255,0.080,0.2050,0.0895,0.0395,0.055,7.0


#### Modelo A
Consideramos 9 parámetros, llamados $\alpha_i$, para el siguiente modelo:
$$ \log(A) = \alpha_0 +  \alpha_1 W_1 + \alpha_2 W_2 +\alpha_3 W_3 +\alpha_4 W_4 + \alpha_5 S + \alpha_6 \log L + \alpha_7 \log D+  \alpha_8 \log H$$

In [3]:
def train_model_A(data):
    y = np.log(data.loc[:, "rings"].values.ravel())
    X = (
        data.assign(
            intercept=1.,
            length=lambda x: x["length"].apply(np.log),
            diameter=lambda x: x["diameter"].apply(np.log),
            height=lambda x: x["height"].apply(np.log),
        )
        .loc[: , ["intercept", "whole_weight", "shucked_weight", "viscera_weight", "shell_weight", "sex", "length", "diameter", "height"]]
        .values
    )
    coeffs = np.linalg.lstsq(X, y, rcond=None)[0]
    return coeffs

def test_model_A(data, coeffs):
    X = (
        data.assign(
            intercept=1.,
            length=lambda x: x["length"].apply(np.log),
            diameter=lambda x: x["diameter"].apply(np.log),
            height=lambda x: x["height"].apply(np.log),
        )
        .loc[: , ["intercept", "whole_weight", "shucked_weight", "viscera_weight", "shell_weight", "sex", "length", "diameter", "height"]]
        .values
    )
    ln_anillos = np.dot(X, coeffs)
    return np.exp(ln_anillos)

#### Modelo B
Consideramos 6 parámetros, llamados $\beta_i$, para el siguiente modelo:
$$ \log(A) = \beta_0 + \beta_1 W_1 + \beta_2 W_2 +\beta_3 W_3 +\beta W_4 + \beta_5 \log( L  D H ) $$

In [4]:
def train_model_B(data):
    y = np.log(data.loc[:, "rings"].values.ravel())
    X = (
        data.assign(
            intercept=1.,
            ldh=lambda x: (x["length"] * x["diameter"] * x["height"]).apply(np.log),
        )
        .loc[: , ["intercept", "whole_weight", "shucked_weight", "viscera_weight", "shell_weight", "ldh"]]
        .values
    )
    coeffs = np.linalg.lstsq(X, y, rcond=None)[0]
    return coeffs

def test_model_B(data, coeffs):
    X = (
        data.assign(
            intercept=1.,
            ldh=lambda x: (x["length"] * x["diameter"] * x["height"]).apply(np.log),
        )
        .loc[: , ["intercept", "whole_weight", "shucked_weight", "viscera_weight", "shell_weight", "ldh"]]
        .values
    )
    ln_anillos = np.dot(X, coeffs)
    return np.exp(ln_anillos)

#### Modelo C
Consideramos 12 parámetros, llamados $\theta_i^{k}$, con $k \in \{M, F, I\}$, para el siguiente modelo:

Si $S=male$:
$$ \log(A) = \theta_0^M + \theta_1^M W_2  + \theta_2^M W_4 + \theta_3^M \log( L  D H ) $$

Si $S=female$
$$ \log(A) = \theta_0^F + \theta_1^F W_2  + \theta_2^F W_4 + \theta_3^F \log( L  D H ) $$

Si $S=indefined$
$$ \log(A) = \theta_0^I + \theta_1^I W_2  + \theta_2^I W_4 + \theta_3^I \log( L  D H ) $$

In [5]:
def train_model_C(data):
    df = (
        data.assign(
            intercept=1.,
            ldh=lambda x: (x["length"] * x["diameter"] * x["height"]).apply(np.log),
        )
        .loc[: , ["intercept", "shucked_weight", "shell_weight", "ldh", "sex", "rings"]]
    )
    coeffs_dict = {}
    for sex, df_sex in df.groupby("sex"):
        X = df_sex.drop(columns=["sex", "rings"])
        y = np.log(df_sex["rings"].values.ravel())
        coeffs_dict[sex] = np.linalg.lstsq(X, y, rcond=None)[0]
    return coeffs_dict

def test_model_C(data, coeffs_dict):
    df = (
        data.assign(
            intercept=1.,
            ldh=lambda x: (x["length"] * x["diameter"] * x["height"]).apply(np.log),
        )
        .loc[: , ["intercept", "shucked_weight", "shell_weight", "ldh", "sex", "rings"]]
    )
    pred_dict = {}
    for sex, df_sex in df.groupby("sex"):
        X = df_sex.drop(columns=["sex", "rings"])
        ln_anillos = np.dot(X, coeffs_dict[sex])
        pred_dict[sex] = np.exp(ln_anillos)
    return pred_dict

### 1. Split Data (1 pto)

Crea dos dataframes, uno de entrenamiento (80% de los datos) y otro de test (20% restante de los datos) a partir de `abalone_data`.

_Hint:_ `sklearn.model_selection.train_test_split` funciona con dataframes!

In [6]:
from sklearn.model_selection import train_test_split

abalone_train, abalone_test = train_test_split(abalone_data, test_size =0.2)
abalone_train 

,sex,length,diameter,height,whole_weight,shucked_weight,viscera_weight,shell_weight,rings
3952,0.0,0.315,0.235,0.080,0.1800,0.0800,0.0450,0.0470,5.0
2550,0.0,0.280,0.220,0.080,0.1315,0.0660,0.0240,0.0300,5.0
2493,0.0,0.420,0.310,0.100,0.2865,0.1150,0.0735,0.0850,8.0
1252,0.0,0.410,0.335,0.110,0.3300,0.1570,0.0705,0.1700,7.0
3275,1.0,0.460,0.355,0.110,0.4150,0.2150,0.0820,0.1300,12.0
98,1.0,0.470,0.370,0.130,0.5225,0.2010,0.1330,0.1650,7.0
3287,1.0,0.550,0.460,0.130,0.7085,0.3050,0.1455,0.2050,12.0
1216,0.0,0.310,0.225,0.070,0.1055,0.4350,0.0150,0.0400,5.0
318,-1.0,0.580,0.455,0.155,0.8365,0.3150,0.1385,0.3200,18.0
3135,1.0,0.460,0.350,0.125,0.5165,0.1885,0.1145,0.1850,9.0


### 2. Entrenamiento (1 pto)

Utilice las funciones de entrenamiento definidas más arriba con tal de obtener los coeficientes para los datos de entrenamiento. Recuerde que para el modelo C se retorna un diccionario donde la llave corresponde a la columna `sex`.

In [7]:
coeffs_A = train_model_A(abalone_train) #obtenemos los valores del coef A,B,C
coeffs_B = train_model_B(abalone_train)
coeffs_C = train_model_C(abalone_train)
coeffs_A

array([ 3.18348105,  0.78639769, -1.73744132, -0.71463129,  0.52672575,
        0.0081293 , -0.09848919,  0.57096449,  0.24568309])

### 3. Predicción (1 pto)

Con los coeficientes de los modelos realize la predicción utilizando el conjunto de test. El resultado debe ser un array de shape `(835, )` por lo que debes concatenar los resultados del modelo C. 

**Hint**: Usar `np.concatenate`.

In [8]:
y_pred_A = test_model_A(abalone_test,coeffs_A) #predecimos los valores del df
y_pred_B = test_model_B(abalone_test, coeffs_B)
y_pred_C = np.concatenate(list(test_model_C(abalone_test, coeffs_C).values()))
#y_pred_C

### 4. Cálculo del error (1 pto)

Se utilizará el Error Cuadrático Medio (MSE) que se define como 

$$\textrm{MSE}(y,\hat{y}) =\dfrac{1}{n}\sum_{t=1}^{n}\left | y_{t}-\hat{y}_{t}\right |^2$$

Defina una la función `MSE` y el vectores `y_test_A`, `y_test_B` e `y_test_C` para luego calcular el error para cada modelo. 

**Ojo:** Nota que al calcular el error cuadrático medio se realiza una resta elemento por elemento, por lo que el orden del vector es importante, en particular para el modelo que separa por `sex`.

In [9]:
def MSE(y_real, y_pred):
    return np.sqrt(((y_pred -y_real)**2).mean())

In [10]:
#dado que buscamos predecir el valor de los anillos según lo estipulado en la clase anterior, es que analizaremos la entrada de estos como valores para luego tomar en cuenta el sexo en la columna del dataframe
y_test_A = abalone_test['rings'].values #obtenemos los valores del df de los "rings"
y_test_B = abalone_test['rings'].values
#sex_list = [-1,0,1] #para tener en orden la lectura de datos 
y_test_C = np.concatenate(list((abalone_test.groupby('sex').get_group(-1)['rings'],abalone_test.groupby('sex').get_group(0)['rings'],abalone_test.groupby('sex').get_group(1)['rings'] )))  #separando por sexo y obtenemos los valores
y_test_C[0]
#y_test_A
#for i in sex_list 

13.0

In [11]:
error_A = MSE(y_test_A,y_pred_A) #evaluamos en la función definida anteriormente
error_B = MSE(y_test_B,y_pred_B)
error_C = MSE(y_test_C,y_pred_C)

In [12]:
print(f"Error modelo A: {error_A:.2f}")
print(f"Error modelo B: {error_B:.2f}")
print(f"Error modelo C: {error_B:.2f}")

Error modelo A: 2.22
Error modelo B: 2.24
Error modelo C: 2.24


**¿Cuál es el mejor modelo considerando esta métrica?**

El mejor modelo considerando como métrica el `MSE` es el modelo **A**, dado que tiene el menor error dentro de los **3**
